### Covid case forecasting Using TimeSeriesModel

#### 1. Import Libraries

In [3]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
import sklearn.metrics as metrics

from arcgis.gis import GIS
from arcgis.learn import TimeSeriesModel, prepare_tabulardata

#### 2. Accessing the Dataset

In [4]:
# Access the data table
data_table = r''

Exception: Could not login using Pro authencation.Please verify in Pro that you are logged in.

#### 3. Raw Data Cleaning

##### 3.1. Calculate Moving Average for Confirmed Cases

##### 3.2. Cut Off first 6 day's date

#### 6. Time Series Data Preprocessing